In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [2]:
# with help from http://planspace.org/20150423-forward_selection_with_statsmodels/
import statsmodels.formula.api as smf

def forward_selected(data, response):
    """Linear model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by adjusted R-squared
    """
    
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response, ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < (best_new_score * 1):
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response, ' + '.join(selected))
    
    model = smf.ols(formula, data).fit()
    return model

In [7]:
# response=pd.read_pickle('data/ZipcodeMVPSnew.pkl')[['RegionName','pct_delta']]
# response['scaled_pct'] = response.pct_delta - 1
# response=response.rename(columns={'RegionName': 'zipcode'})

### Changed this to adjust for inflation

In [17]:
response=pd.read_pickle('data/ZipcodeMVPSnew.pkl')[['RegionName','2000_agg','2010_agg']]
response=response.rename(columns={'RegionName': 'zipcode'})
response['2010_agg_adj'] = response['2010_agg']/1.26
response['pct_delta'] = (response['2010_agg_adj']/response['2000_agg'])/100
response = response.drop('2010_agg', 1)

In [18]:
response.head(3)

,zipcode,2000_agg,2010_agg_adj,pct_delta
1,79936,55.666667,65.343915,0.011738
2,60629,98.666667,107.142857,0.010859
3,90650,133.333333,190.211640,0.014266


In [18]:
features=pd.read_pickle('data/all_features_norm_not_transformed.pkl')
print features.shape
features.head(10)

(27653, 64)


,A001,A002,A003,A004,A005,A006,A007,A008,A009,A010,A011,A012,A013,A014,A015,A016,A017,A018,A019,A020,A021,A022,A023,A024,A025,A026,A027,A028,A029,A030,A031,A032,A033,A034,A035,A036,A037,A038,A039,A040,A041,A042,A043,A044,A045,A046,A047,A048,A049,A050,A051,A052,A053,A054,A055,A056,n_establishments,paid_employees,first_quarter_payroll_1000,annual_payroll_1000,prison,jail,median_income,zipcode
0,0.403822,-0.839925,-0.577200,-0.168204,-0.114910,0.097627,-0.022710,0.164482,0.297415,1.855374,-0.654254,-0.783044,-0.641046,-0.195054,-0.049217,-0.050336,-0.098035,-0.050738,-0.022365,1.082934,0.719438,-0.767238,-0.457822,-0.048472,-0.167051,0.288460,0.114242,0.384865,0.639734,2.118527,0.535081,-0.394929,-0.177156,-0.011208,-0.131380,-0.366799,-0.888653,-0.313466,-0.455594,-0.169235,-0.295995,1.116199,0.259922,0.434214,1.593216,1.397376,-0.743050,0.584263,0.759381,-0.798737,-0.134993,-0.549322,-0.651977,-0.223378,0.495932,-1.124265,0.463735,0.737979,0.231006,0.318245,-0.172158,-0.323992,0.332304,01001
1,1.261860,-1.594272,1.084305,3.638466,-1.346952,-2.015987,-1.228271,-0.871073,-0.804575,-0.223704,-0.396719,-1.489846,0.510741,2.836251,-1.107246,-1.880773,-1.383069,-0.897923,-0.802669,-0.377333,0.461899,-1.503218,1.457233,3.958870,-1.254782,-1.507307,-0.860163,-0.722090,-0.705649,-0.149133,-0.228580,-0.192467,-0.162902,1.520559,-0.131380,-0.062326,-1.609325,-1.104775,-0.981585,-0.116915,0.173738,0.114255,-0.327087,-0.496995,0.519232,0.092856,-0.839470,-0.632176,0.752600,-0.791203,-0.255063,-0.549322,-0.163923,-0.324466,0.984798,-0.027929,0.766968,1.620780,0.316380,0.397244,-0.172158,-0.323992,0.109920,01002
2,-0.043154,-3.538166,8.003111,6.934284,-3.589901,-4.975048,-4.270877,-2.815896,-2.216499,-1.486001,0.311503,-3.392777,6.712670,5.621108,-2.742381,-4.054417,-3.798934,-2.458525,-1.973126,-1.322211,-0.246334,-3.371474,7.949734,7.254629,-3.622197,-4.355765,-4.054045,-2.835367,-2.179163,-1.396346,-0.272079,-0.122871,-0.191410,1.766398,-0.131380,-0.247657,-2.481250,-2.529132,-2.121233,-1.878365,-1.705193,-0.706614,1.535146,-1.686872,-2.722984,-2.143463,-1.235864,-3.293888,1.064512,-1.137736,-0.975481,-1.112115,1.602369,1.267669,-0.808171,1.557843,-0.610010,-0.496545,-0.289793,-0.339523,-0.172158,-0.323992,-2.281652,01003
3,-0.412991,-0.317685,0.836319,-0.332995,-0.272864,0.661258,0.551366,-0.315410,-0.666826,0.073308,0.161274,-0.293719,1.130934,-0.268988,-0.385863,0.407274,0.467380,-0.006150,-0.477543,-0.205536,-0.096103,-0.257713,0.429643,-0.348087,-0.103067,0.721921,0.547311,-0.571142,-0.833780,0.191016,0.665580,-0.432891,-0.177156,-0.313779,-0.131380,-0.466084,0.196804,0.161320,0.245728,-0.134355,-0.061129,-0.223750,-0.144912,0.175545,-0.230531,0.092856,0.296146,-0.174505,0.603425,-0.625471,-0.455179,0.254667,0.231169,0.231517,-0.456070,-0.439055,-0.411169,-0.392766,-0.256765,-0.299852,-0.172158,-0.323992,0.979231,01005
4,0.255692,0.175542,0.265952,-0.431869,0.011454,1.295342,0.752293,-0.365925,-0.735700,-0.557841,-0.246490,0.195607,0.244944,-0.342922,-0.193494,0.693279,0.570183,-0.362859,-0.802669,-0.463231,0.311668,0.138583,0.242808,-0.460442,0.344822,1.464998,0.817979,-0.369877,-0.641583,-0.545973,0.544747,-0.420237,-0.191410,-0.011208,-0.005316,-0.413132,0.321364,-0.682743,-0.455594,-0.099475,0.203096,-0.501397,-0.650953,-0.496995,-0.169739,-0.341983,0.542554,-0.836923,0.779723,-0.821337,-0.655295,-0.227727,0.254410,0.155701,0.199262,-0.478210,-0.058225,-0.281321,-0.220885,-0.250865,-0.172158,-0.323992,1.037927,01007
5,-0.679626,-0.723872,-0.403610,-0.332995,-0.462409,0.731712,1.039331,1.124264,-0.425766,-0.409335,0.397348,-0.674305,-0.020853,-0.293633,-0.289678,0.064067,1.238401,1.242332,-0.412517,0.138056,-0.332180,-0.710624,-0.738074,-0.310635,-0.550956,1.217306,0.655578,0.837710,-0.449385,-0.659356,0.767079,-0.489833,-0.177156,-0.351600,-0.131380,-0.499179,0.437028,-0.629989,-1.244581,-0.762198,-0.883161,-0.597970,0.178955,0.072077,-1.182932,-0.838943,-0.164528,-0.234725,0.101655,-0.068005,-0.975481,0.254667,-0.001238,-0.248650

In [21]:
merged=pd.merge(response, features, on=['zipcode'])
merged=merged.astype('float')
merged=merged.rename(columns={'scaled_pct' : 'response'})
merged.head(10)

,zipcode,pct_delta,response,A001,A002,A003,A004,A005,A006,A007,A008,A009,A010,A011,A012,A013,A014,A015,A016,A017,A018,A019,A020,A021,A022,A023,A024,A025,A026,A027,A028,A029,A030,A031,A032,A033,A034,A035,A036,A037,A038,A039,A040,A041,A042,A043,A044,A045,A046,A047,A048,A049,A050,A051,A052,A053,A054,A055,A056,n_establishments,paid_employees,first_quarter_payroll_1000,annual_payroll_1000,prison,jail,median_income
0,79936,1.479042,0.479042,6.994015,1.045942,1.381888,0.375606,0.611679,0.238535,-0.711602,-1.275192,-1.286695,-0.929105,-0.418180,0.956779,1.263832,0.223907,0.143151,-0.221939,-0.920457,-1.210043,-1.257847,-0.892721,0.483360,1.044405,1.270398,0.513305,1.048648,0.598075,-0.372961,-1.174935,-1.154109,-0.886122,0.003418,-0.350641,-0.120142,-0.219225,-0.005316,5.126962,1.255569,0.477844,0.421058,1.609655,1.847160,-1.358482,-1.076028,-1.169534,-1.203196,-1.366963,2.235265,-1.402988,0.861091,-0.911737,-0.575249,-0.388525,1.648851,1.469845,5.325187,1.068408,3.563179,4.055737,1.079163,1.354307,-0.172158,-0.323992,-0.054208
1,60629,1.368243,0.368243,7.191755,1.713249,1.059507,0.705188,1.180314,-0.571684,-1.371790,-1.426737,-1.321132,-0.817726,-0.096261,1.609212,0.865137,0.494999,0.912627,-0.507945,-1.331668,-1.299220,-1.257847,-0.720925,0.161436,1.667157,1.130272,0.887823,1.176617,-0.454616,-1.184964,-1.325883,-1.218175,-0.716048,-2.007232,0.946383,-0.105888,-0.313779,-0.131380,3.862735,0.908579,2.060462,1.473041,2.063098,1.817802,-0.935975,-0.752161,-0.496995,-0.818182,-0.870003,2.363825,-0.873054,0.291514,-0.278938,0.745518,0.254667,2.927088,3.137795,5.062190,1.577421,1.283957,0.677465,0.336771,0.409415,-0.172158,-0.323992,-0.454877
2,90650,1.797500,0.797500,6.607130,0.581728,0.836319,0.573355,0.832815,-0.184188,-0.941233,-1.022618,-0.804575,-0.483588,-0.053338,0.630562,0.820837,0.445710,0.575981,-0.164738,-0.971858,-0.987100,-0.867695,-0.463231,0.118513,0.534880,0.709894,0.625660,0.920680,-0.145001,-0.751896,-0.873038,-0.641583,-0.432590,-1.601235,-0.230429,-0.048874,1.860952,0.372878,4.054685,1.362335,2.060462,1.034714,1.365494,0.584754,-1.599914,-1.440378,-0.807397,-1.243723,-0.807883,2.020998,0.078417,0.833968,-0.881603,-0.455179,-0.388525,3.066532,2.783988,4.040425,2.282208,1.892909,1.825187,0.682492,0.808127,-0.172158,-0.323992,0.360311
3,90201,1.753846,0.753846,6.308773,1.742262,1.406687,0.754625,1.148723,-0.395550,-1.630125,-1.553024,-1.390007,-1.003358,0.032507,1.663582,1.219533,0.544289,0.864535,-0.279140,-1.485872,-1.432986,-1.322873,-0.892721,0.032667,1.667157,1.363815,0.887823,1.176617,-0.392693,-1.455632,-1.476832,-1.282241,-0.942814,-1.528736,-0.445545,-0.063127,-0.294868,-0.005316,5.696195,1.744914,3.643081,3.401675,2.010777,2.199460,-2.070707,-1.622553,-1.117800,-1.851639,-1.429083,3.456588,-1.150065,0.833968,-0.881603,-0.134993,-0.790519,3.833475,3.289427,3.497682,3.241502,1.460428,1.325463,0.438963,0.529263,-0.172158,3.086496,-0.589723
4,90280,1.704663,0.704663,5.827838,1.307062,1.158701,0.738146,1.022360,-0.360323,-1.314383,-1.249935,-1.217821,-0.891979,-0.160645,1.228626,0.909436,0.494999,0.720258,-0.279140,-1.383069,-1.210043,-1.192822,-0.806823,0.225821,1.270860,1.223689,0.925275,1.112633,-0.330770,-1.076697,-1.174935,-1.090044,-0.829431,-1.548069,-0.451872,-0.077381,-0.257047,-0.005316,5.689576,1.664840,2.588002,2.262027,1.644535,1.436144,-1.937919,-1.602311,-1.014333,-1.628736,-1.211663,2.974487,-0.656263,0.813626,-0.859003,-0.215040,-0.549322,3.577828,3.137795,3.362989,2.693334,1.671698,1.559717,0.730275,0.949190,-0.172158,-0.323992,-0.435488
5,60618,1.607803,0.607803,5.666292,0.668768,-0.750790,1.199561,2.854628,-0.113734,-1.371790,-1.224678,-1.080072,-0.780599,0.053968,0.521823,-0.729645,0.840026,2.307301,0.121268,-1.228865,-1.165454,-1.127796,-0.720925,0.011205,0.704722,-0.644656,1.412148,2.712237,-0.330770,-1.293232,-1.124619,-0.897846,-0.716048,-0.808575,-0.287371,-0.105888,0.593935,-0.131380,2.485985,-0.924242,0.741613,-0.017268,0.266767,0.115021,0.403974,0.320647,-0.755664,0.357121,-0.870003,0.01

In [ ]:
#minimerge = merged[0:50]
bestmodel = forward_selected(merged, 'response')

In [11]:
bestmodel.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               response   R-squared:                       0.356
Model:                            OLS   Adj. R-squared:                  0.354
Method:                 Least Squares   F-statistic:                     159.7
Date:                Sun, 06 Dec 2015   Prob (F-statistic):               0.00
Time:                        22:50:30   Log-Likelihood:                -177.50
No. Observations:               11602   AIC:                             437.0
Df Residuals:                   11561   BIC:                             738.7
Df Model:                          40                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------------------
Intercept                      0.4691      0.004    130.389      0.000         0.462     0.476
A002                          -0.0629      0.010     -6.366      0.000        -0.082    -0.044
A054                           0.0174      0.012      1.447      0.148        -0.006     0.041
A003                          -0.0822      0.009     -9.125      0.000        -0.100    -0.065
median_income                  0.0769      0.004     17.849      0.000         0.068     0.085
A050                           0.1111      0.005     24.278      0.000         0.102     0.120
A052                          -0.0779      0.004    -19.507      0.000        -0.086    -0.070
A024                          -0.1281      0.087     -1.479      0.139        -0.298     0.042
A051                          -0.0577      0.004    -13.904      0.000        -0.066    -0.050
A040                           0.0867      0.014      6.190      0.000         0.059     0.114
A032                          -0.0846      0.013     -6.663      0.000        -0.110    -0.060
A055                          -0.0574      0.011     -5.007      0.000        -0.080    -0.035
A031                          -0.0962      0.016     -6.064      0.000        -0.127    -0.065
A044                           0.0586      0.011      5.393      0.000         0.037     0.080
A006                           0.0085      0.013      0.629      0.529        -0.018     0.035
A034                          -0.0217      0.005     -4.368      0.000        -0.031    -0.012
A035                           0.0117      0.003      4.622      0.000         0.007     0.017
A036                           0.0127      0.005      2.383      0.017         0.002     0.023
annual_payroll_1000           -0.0300      0.019     -1.560      0.119        -0.068     0.008
n_establishments               0.0527      0.006      8.982      0.000         0.041     0.064
paid_employees                -0.0388      0.008     -4.713      0.000        -0.055    -0.023
A042                          -0.1591      0.018     -8.887      0.000        -0.194    -0.124
A037                          -0.2138      0.018    -11.871      0.000        -0.249    -0.178
A041                          -0.0377      0.013     -3.006      0.003        -0.062    -0.013
A025                           0.0287      0.006      4.553      0.000         0.016     0.041
A048                           0.1782      0.016     10.909      0.000         0.146     0.210
A029                          -0.0610      0.010     -5.809      0.000        -0.082    -0.040
A056                          -0.0314      0.006     -5.056      0.000        -0.044    -0.019
A046                          -0.0665      0.011     -5.985      0.000        -0.088    -0.045
A047                           0.0892      